In [11]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.signal as ss
from statistics import median, mean

# import step_detect
# from bokeh.plotting import figure, show
# from bokeh.io import output_notebook#, gridplot
# from scipy.ndimage.filters import gaussian_filter1d

In [2]:
#functino for low pass filter
from scipy.signal import butter, lfilter, freqz

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [3]:
#function for calculate the time interval between peaks
def freqCal(df, peaksIndex):
    dfTime = df.iloc[peaksIndex] #get sub dataframe 
    prev = dfTime[0: 1] #cache the first 
    for i, _ in dfTime.iteritems():
        cur = dfTime[i] #cache cur
        dfTime[i] = abs(dfTime[i] - prev) #get difference
        prev = cur #set prev from cur (cur val we've cached before)
    return dfTime

In [4]:
#function for reading database, return a dataframe, return a dataframe
def readDB(csv_database):
    sql = 'select * from sensorProcessed' #sql need to be executed. 
    df = pd.read_sql_query(sql, csv_database) # read data as dataframe
    return df

In [5]:
#set credentials file for plotly(username & API key)
import plotly
username = 'mengyizhou95' #username registered on Ploty
api_key = 'X5DOiJATuUqPbsabrbxA' #the api key plotly asigned
plotly.tools.set_credentials_file(username=username, api_key=api_key)

In [6]:
#set the conversion ratio from min to ms
#set the conversion ratio from min to ms
timeConversionVal = 60000
#set low pass filter parameters
order = 5 #order of filter (1-)
fs = 50       # sample rate, Hz
cutoff = 10 # desired cutoff frequency of the filter, Hz

sizeMag = 2.5 # set the size of range in histogram of magnitude
sizeInter = 5000 # set the size of range in histogram of time interval

h = 5

In [7]:
#function for trans time from min to ms
def MinToMS(time):
    time = time * timeConversionVal
    time = df.time.iloc[(df['time']-time).abs().argsort()[:1]] #find the closest time on table
    return time.values[0]

In [8]:
#function for debouncing orientation
def orientationTrans(dfOrientation, start, end, threshold):
    startIndex = df.index[df['time'] == start].tolist()[0] #get the index
    endIndex = df.index[df['time'] == end].tolist()[0]
    prev = dfOrientation[startIndex]
    
    for i in range(startIndex, endIndex+1):
        diff = dfOrientation[i] - prev
        prev = dfOrientation[i]
        if(diff > threshold or diff < -threshold):
            if(diff > 0):
                dfOrientation[i] = threshold * 2 - diff
            else:
                dfOrientation[i] = -(threshold * 2 + diff)
        else:
            dfOrientation[i] = diff
    return dfOrientation

In [9]:
def mergeClosePoints(list, threshold):
    res = []
    i = 0
    j = 1
    while(i < len(list)):
        if j < len(list) and list[j] - list[j-1] <= threshold:
            j += 1
            continue
        else:
            res.append(mean(list[i:j]))
            i = j
            j += 1
            
    return res

In [10]:
list = [1, 10, 11, 12, 20, 21, 70, 71, 72]
res = mergeClosePoints(list, 5)
# print(res)

In [11]:
#function for generating orientation chart
def findCandidates(df, start, end):
    start = MinToMS(start)
    end = MinToMS(end)

    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    dfOrientationX = orientationTrans(df.orientationX, start, end, 180)
    dfOrientationY = orientationTrans(df.orientationY, start, end, 90)

    for i in range(len(dfOrientationX)):
        if abs(dfOrientationX[i]) <= 5:
            dfOrientationX[i] = 0
        if abs(dfOrientationY[i]) <= 5:
            dfOrientationY[i] = 0
#     for i in range(len(dfOrientationY)):
#         if abs(dfOrientationY[i]) <= 5:
#             dfOrientationY[i] = 0
    
#     peaksPosIndex, _ = ss.find_peaks(dfOrientationX[startIndex: endIndex], height = 5)
#     peaksNegIndex, _ = ss.find_peaks(-dfOrientationX[startIndex: endIndex], height = 5)
    
#     print(peaksPosIndex)
#     print(peaksNegIndex)
    
#     trace2 = go.Scattergl(
#         x=df.time[peaksPosIndex]/timeConversionVal,
#         y=dfOrientationX[peaksPosIndex],
#         mode = 'markers',
#         name = 'peaks',
#         marker = dict(
#             color = ('rgb(255, 0, 0)'))
#     )
       
#     trace3 = go.Scattergl(
#         x=df.time[peaksNegIndex]/timeConversionVal,
#         y=dfOrientationX[peaksNegIndex],
#         mode = 'markers',
#         name = 'peaks',
#         marker = dict(
#             color = ('rgb(127, 127, 127)'))
#     )
    list = []
    print(list)
    res = []
    for i in range(startIndex, endIndex+1):
        if dfOrientationX[i] != 0:
            list.append(df.time[i])
    return list    
            
        
#     trace0 = go.Scattergl(
#         x=df.time[startIndex: endIndex]/timeConversionVal,
#         y=dfOrientationX[startIndex: endIndex],
#         name = 'orientationX',
#         line = dict(
#             color = ('rgb(255, 0, 0)'),
#             width = 1)
#     )
#     trace1 = go.Scattergl(
#         x=df.time[startIndex: endIndex]/timeConversionVal,
#         y=dfOrientationY[startIndex: endIndex],
#         name = 'orientationY',
#         line = dict(
#             color = ('rgb(0, 0, 255)'),
#             width = 1)
#     )

#     data = [trace0, trace1]#, trace2, trace3]
#     layout = dict(title = 'orientation sensor',
#                        xaxis = dict(title = 'Time'),
#                        yaxis = dict(title = 'orientation sensor output'))

#     fig = dict(data = data, layout = layout)
#     plotly.offline.plot(fig, filename='orientation sensor test.html')

In [12]:
def intervalSelect(df, start, end, peaksInRange, dfAcc):
     #trans time to ms
    start = MinToMS(start)
    end = MinToMS(end) 
    #calculate index from time
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    #low pass filter
    processedSignal = butter_lowpass_filter(dfAcc[startIndex: endIndex], cutoff, fs, order)
    #peaks index

    peaksPosIndex, _ = ss.find_peaks(processedSignal, height = h, distance = 20)
    peaksNegIndex, _ = ss.find_peaks(-processedSignal, height = h, distance = 20) 
         
    indexIntervalPos = []
    indexIntervalNeg = []
    for i in range(1, len(peaksPosIndex)):
        cur = peaksPosIndex[i] - peaksPosIndex[i-1]
        indexIntervalPos.append(cur)
    for i in range(1, len(peaksNegIndex)):
        cur = peaksNegIndex[i] - peaksNegIndex[i-1]
        indexIntervalNeg.append(cur)
    
    indexMeanPos = mean(indexIntervalPos)
    indexMeanNeg = mean(indexIntervalNeg)
    indexMean = (indexMeanPos + indexMeanNeg) / 2
    return (int)(indexMean * peaksInRange)

In [13]:
def test(df, start, end, dfAcc):
    
    res = []
    
    threshold = 0.96
    
    indexInterval = intervalSelect(df, 10, 160, 100, dfAccX)
    
    start = MinToMS(start)
    end = MinToMS(end) 
    
    #calculate index from time
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    #low pass filter
    processedSignal = butter_lowpass_filter(dfAcc[startIndex: endIndex], cutoff, fs, order)
    #peaks index

    peaksPosIndex, _ = ss.find_peaks(processedSignal)#, height = h)
    peaksNegIndex, _ = ss.find_peaks(-processedSignal)#, height = h) 
    
    peaksPosIndexPlus = []
    peaksNegIndexPlus = []
    # index for Time 
    #time index is seperated from peak index. Because peak index from 0 after we do low pass filtering. But time depends on your input
    for i in range(len(peaksPosIndex)):
        peaksPosIndexPlus.append(peaksPosIndex[i] + startIndex)
    for i in range(len(peaksNegIndex)):
        peaksNegIndexPlus.append(peaksNegIndex[i] + startIndex)
    
####################################magnitude abnormal detection################################
    count = startIndex
    while(count < endIndex):
        s = count
        e = count + indexInterval
    #         print('s:' + str(s))
    #         print('e:' + str(e))
        processedSignal = butter_lowpass_filter(dfAcc[s: e], cutoff, fs, order)
        #peaks index
       
        peaksPosIndex, _ = ss.find_peaks(processedSignal)#, height = h, distance = 20)
        peaksNegIndex, _ = ss.find_peaks(-processedSignal)#, height = h, distance = 20)

        peaksPosIndexPlus = []
        peaksNegIndexPlus = []
        # index for Time 
        #time index is seperated from peak index. Because peak index from 0 after we do low pass filtering. But time depends on your input
        for i in range(len(peaksPosIndex)):
            peaksPosIndexPlus.append(peaksPosIndex[i] + s)
        for i in range(len(peaksNegIndex)):
            peaksNegIndexPlus.append(peaksNegIndex[i] + s)

        sortedProcessedSignalPos = sorted(processedSignal[peaksPosIndex])
        sortedProcessedSignalNeg = sorted(processedSignal[peaksNegIndex])
        
        medianPos = median(sortedProcessedSignalPos)
        medianNeg = median(sortedProcessedSignalNeg)
        avgPos = mean(sortedProcessedSignalPos)
        avgNeg = mean(sortedProcessedSignalNeg)
        #get threshold of peaks magnitude
        thresholdPos = sortedProcessedSignalPos[(int)(threshold * len(sortedProcessedSignalPos))]
        thresholdNeg = sortedProcessedSignalNeg[(int)((1-threshold) * len(sortedProcessedSignalNeg))]
        
        errorPercentage = 0.1
        #calcute index of time
        posIndexTime = [i for i in peaksPosIndexPlus if processedSignal[i - s] > thresholdPos and abs(processedSignal[i-s] - medianPos) > errorPercentage * medianPos]
        negIndexTime = [i for i in peaksNegIndexPlus if processedSignal[i - s] < thresholdNeg and abs(processedSignal[i-s] - medianNeg) > abs(errorPercentage * medianNeg)]

        #caculate the index of peaks
        posIndexSignal = [i-s for i in posIndexTime]
        negIndexSignal = [i-s for i in negIndexTime]
        ##################### TIME INTERVAL #################################
        dfPosTime = freqCal(dfAccTime, peaksPosIndexPlus)
        dfNegTime = freqCal(dfAccTime, peaksNegIndexPlus)
        sortedTimePos = sorted(dfPosTime)
        sortedTimeNeg = sorted(dfNegTime)
        
        medianPosInterval = median(sortedTimePos)
        medianNegInterval = median(sortedTimeNeg)
        
        thresholdPosInterval = sortedTimePos[(int)((1-threshold) * len(sortedTimePos))] #calculate the median of time interval
        thresholdNegInterval = sortedTimeNeg[(int)((1-threshold) * len(sortedTimeNeg))]
        
        posIndexTimeInterval = []
        negIndexTimeInterval = []
        
        for i in range(len(peaksPosIndexPlus)):
            diff = abs(dfAccTime[peaksPosIndexPlus[i]] - dfAccTime[peaksPosIndexPlus[i-1]])
            if  diff < thresholdPosInterval and diff < (1 - errorPercentage) * medianPosInterval:
                posIndexTimeInterval.append(peaksPosIndexPlus[i])
        for i in range(len(peaksNegIndexPlus)):
            diff = abs(dfAccTime[peaksNegIndexPlus[i]] - dfAccTime[peaksNegIndexPlus[i-1]])
            if  diff < thresholdNegInterval and diff < (1 - errorPercentage) * medianNegInterval:
                negIndexTimeInterval.append(peaksNegIndexPlus[i])
        
        posIndexSignalInterval = [i-s for i in posIndexTimeInterval]
        negIndexSignalInterval = [i-s for i in negIndexTimeInterval]
        
        res.extend(dfAccTime[posIndexTime].values.tolist())
        res.extend(dfAccTime[negIndexTime].values.tolist())
        res.extend(dfAccTime[posIndexTimeInterval].values.tolist())
        res.extend(dfAccTime[negIndexTimeInterval].values.tolist())
        
        count = count + indexInterval
        
    return sorted(res)

In [14]:
def anomalyCountAcc(candidateOrientation, candidateAcc):
    counts = []
    for timePoint in candidateOrientation:
        lower = timePoint - 500
        upper = timePoint + 500
        count = 0
        for candidate in candidateAcc:
            if(candidate <= upper and candidate >= lower):
#             newRes.append(timePoint)
                count += 1
            else:
                if(candidate > upper):
                    break
                if(candidate < lower):
                    continue
        counts.append(count) 
    return counts    

In [15]:
#execute read database
# csv_database = 'sqlite:///SouthHarrison.db'
# df = readDB(csv_database)
# dfAccTime = df['time']
# dfAccX = df['accX']
# dfAccY = df['accY']
# dfAccZ = df['accZ']

# start = 10
# end = 180
# candidateAccX = test(df, start, end, dfAccX)
# candidateAccY = test(df, start, end, dfAccY)
# candidateAccZ = test(df, start, end, dfAccZ)

# candidateOrientation = findCandidates(df, start, end)
# candidateOrientation = mergeClosePoints(candidateOrientation, 1000)
# # print(candidateOrientation)

# countsAccX = anomalyCountAcc(candidateOrientation, candidateAccX)
# countsAccY = anomalyCountAcc(candidateOrientation, candidateAccY)
# # countsAccZ = anomalyCountAcc(candidateOrientation, candidateAccZ)
# # print(countsAccX)
# # print(countsAccY)
# # print(countsAccZ)
# countsAcc = [x + y for x, y in zip(countsAccX, countsAccY)]

# # data = {'countsOfAccX':countsAccX, 'countsOfAccY':countsAccY, 'countsOfAccZ':countsAccZ}
# data = {'countsOfAcc':countsAcc}
# dfTimeCount = pd.DataFrame(data, index=candidateOrientation)
# print(dfTimeCount)


In [16]:
for index, row in dfTimeCount.iterrows():
    if(row['countsOfAcc'] == 0):
        dfTimeCount.drop([index], inplace = True)
# print(dfTimeCount)

NameError: name 'dfTimeCount' is not defined

In [ ]:
def mergeTurning(df, threshold):
    time = []
    counts = []
    i = 0
    j = 1
    while(i < len(df.index)):
        if j < len(df.index) and df.index[j] - df.index[j-1] <= threshold:
            j += 1
            continue
        else:
            time.append(mean(df.index[i: j]))
            counts.append(sum(df.countsOfAcc[i: j]))
            i = j
            j += 1
            
    return time, counts

In [ ]:
timeTurning, countsTurning= mergeTurning(dfTimeCount, 30000)

In [23]:
#chart for 
def turningDetection(time, counts):#df):
    trace0 = go.Scattergl(
        x=[i / timeConversionVal for i in time ],#df.index/timeConversionVal,
        y=counts,#df.countsOfAcc,
        mode = 'markers',
        name = 'possible turning',
        marker = dict(
            color = ('rgb(255, 0, 0)'))
    )
    
    data = [trace0]
    
    layout = dict(title = 'turning detection',
                       xaxis = dict(title = 'Time', 
                                        range = [10, 180]),
                       yaxis = dict(title = 'possible points'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename='turning detection.html')

# # turningDetection(dfTimeCount)
# turningDetection(timeTurning, countsTurning)
# print(dfTimeCount.nlargest(10, 'countsOfAcc'))


In [46]:
#function for generating orientation chart
def orientationChartOriginal(df, start, end):
    start = MinToMS(start)
    end = MinToMS(end)

    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    dfOrientationX = df.orientationX
    dfOrientationY = df.orientationY
    dfOrientationZ = df.orientationZ

    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=dfOrientationX[startIndex: endIndex],
        name = 'orientationX',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=dfOrientationY[startIndex: endIndex],
        name = 'orientationY',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=dfOrientationZ[startIndex: endIndex],
        name = 'orientationZ',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
        
    sigma = 10
    trace4 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=sp.ndimage.filters.gaussian_filter1d(dfOrientationX[startIndex: endIndex], sigma),
        name = 'orientationX',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )

    data = [trace0, trace4, trace1]#, trace2]
    layout = dict(title = 'orientation sensor',
                       xaxis = dict(title = 'Time'),
                       yaxis = dict(title = 'orientation sensor output'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename='orientation sensor original.html')
df = readDB(csv_database)
orientationChartOriginal(df, 15, 170)

In [52]:
def edgeDetection(df, start, end):
    
    start = MinToMS(start)
    end = MinToMS(end)
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    sigma = 50
    dfOrientationX = sp.ndimage.filters.gaussian_filter1d(df.orientationX[startIndex: endIndex], sigma)#.values.tolist()
    dfTime = df.time[startIndex: endIndex].values.tolist()
#     print(len(dfOrientationX))
#     print(len(dfTime))
    mask = [-1, -1/2, 0, 1/2, 1] 
    orientCon = np.convolve(dfOrientationX, mask)
    print(len(orientCon))
    trace0 = go.Scattergl(
        x = [x / timeConversionVal for x in dfTime],
        y = orientCon,
        name = 'test',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1
        )
    )
    
#     peaksPosIndex, _ = ss.find_peaks(orientCon, height = 1)
#     peaksNegIndex, _ = ss.find_peaks(-orientCon, height = 1)
#     trace1 = go.Scattergl(
#         x=[dfTime[i]/timeConversionVal for i in peaksPosIndex],
#         y=[orientCon[i] for i in peaksPosIndex],
#         mode = 'markers',
#         name = 'peaks',
#         marker = dict(
#             color = ('rgb(127, 127, 127)'))
#     )
    
#     trace2 = go.Scattergl(
#         x=[dfTime[i]/timeConversionVal for i in peaksNegIndex],
#         y=[orientCon[i] for i in peaksNegIndex],
#         mode = 'markers',
#         name = 'peaks',
#         marker = dict(
#             color = ('rgb(127, 127, 127)'))
#     )
    
    data = [trace0]#, trace1, trace2]
    layout = dict(title = 'test',
                  xaxis = dict(title = 'Time'),
                  yaxis = dict(title = 'orientation Con'))
    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename='test orientationl.html')
    return orientCon, dfTime
    
csv_database = 'sqlite:///SouthHarrison.db'
df = readDB(csv_database)
orient, time = edgeDetection(df, 15, 170)
# peaksPosIndex, _ = ss.find_peaks(orient)

    

335248


In [54]:
list = []
for i in range(len(time)):
    if abs(orient[i]) > 2:
        list.append(time[i]/timeConversionVal)
print(list)

[14.999916666666667, 15.000333333333334, 15.0007, 15.001066666666667, 35.47215, 35.47253333333333, 35.47408333333333, 35.474516666666666, 35.47605, 35.476416666666665, 35.47683333333333, 35.47721666666666, 35.477583333333335, 35.477983333333334, 35.47838333333333, 35.47875, 35.47911666666667, 35.47956666666666, 35.47995, 35.48043333333333, 35.48085, 35.481233333333336, 35.4816, 35.48201666666667, 35.4824, 35.48276666666667, 35.48316666666667, 35.48356666666667, 35.48393333333333, 35.48433333333333, 35.4847, 35.48506666666667, 35.485483333333335, 35.4859, 35.486266666666666, 35.48665, 35.487066666666664, 35.487433333333335, 35.48781666666667, 35.488216666666666, 35.48858333333333, 35.48895, 35.48936666666667, 35.48975, 35.490116666666665, 35.49048333333333, 35.49093333333333, 35.4913, 35.4917, 35.4921, 35.492466666666665, 35.49288333333333, 35.49328333333333, 35.49368333333334, 35.49405, 35.49443333333333, 35.4948, 35.4952, 35.49556666666667, 35.495983333333335, 35.49635, 35.49675, 35.4